In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline

## Example with generated data

Problem: customer segmentation for a marketing campaign of an app. For a group of 20 people, we have their age and their weekly hours of app usage

In [ ]:
age = [20, 22, 24, 24, 25, 28, 29, 34, 36, 38, 39, 41, 42, 44, 56, 57, 61, 62, 63, 63]
hours = [12, 10, 11.2, 12.3, 12.4, 13.5, 14, 8, 7, 6, 7.2, 5.6, 6.7, 6.2, 5, 4.2, 3, 2, 1]

In [ ]:
df = pd.DataFrame(zip(age, hours), columns = ['age', 'hours'])
df.head()

In [ ]:
sns.scatterplot(x = df['age'], y = df['hours'])

#### Using two clusters

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

In [ ]:
# Predicting / assigning the clusters:
df_cl = df
#kmeans.predict(df)
df_cl['clusters'] = kmeans.predict(df)
df_cl


In [ ]:
# Check the size of the clusters
print(df_cl['clusters'].value_counts().sort_index())

In [ ]:
clusterval = np.unique(df['clusters'])
print(clusterval)
for cluster in clusterval:
    sns.scatterplot(x = df_cl[df_cl['clusters']==cluster]['age'], y = df_cl[df_cl['clusters']==cluster]['hours'])
plt.show()

#### Using different number of clusters

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

df_cl['clusters'] = kmeans.predict(df)

clusterval = np.unique(df['clusters'])
for cluster in clusterval:
    sns.scatterplot(x = df_cl[df_cl['clusters']==cluster]['age'], y = df_cl[df_cl['clusters']==cluster]['hours'])
plt.show()

## Using data

In [ ]:
from sklearn import datasets

data = datasets.load_wine()
print(data.keys())
print(data['DESCR'])

In [ ]:
X = pd.DataFrame(data["data"], columns=data["feature_names"])
#this data ALSO has a target variable for supervised learning problems, but this is not what we are interested in here!


In [ ]:
X.head()

In [ ]:
# The scale of "proline" is much higher than the scale of many other variables!
# K-Means is a distance based algorithm: we need to scale / normalize:
from sklearn.preprocessing import StandardScaler
X_prep = StandardScaler().fit_transform(X)

# Now, all features will have the same weight.
pd.DataFrame(X_prep, columns = X.columns).head()

In [ ]:
round(pd.DataFrame(X_prep).describe())

#### We will run a K-Means clustering with 8 clusters (random choice).

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

For best practice, we will also specifiy a value for random_state. Specifying this is just used for reproducing the same results on every run - since the K-Means algorithm is stochastic, we will otherwise get slightly different results each time even if parameters are the same, so fixing it makes it easier to compare. 

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8, random_state = 1234)
kmeans.fit(X_prep)

In [ ]:
# Predicting / assigning the clusters:
clusters = kmeans.predict(X_prep)

# Check the size of the clusters
print(pd.Series(clusters).value_counts().sort_index())

# Explore the cluster assignment in the original dataset
X_df = pd.DataFrame(X)
X_df["cluster"] = clusters
X_df.head(20)

#### Basic Performance value: 'inertia'.
'Inertia' is the mean squared distance between each instance and its closest centroid.

In [ ]:
kmeans.inertia_

### Choosing a value for K

Elbow plot using inertia

In [ ]:
K = range(1, 9)
print(list(K))
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_prep)
    inertia.append(kmeans.inertia_)

print(inertia)

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

There seems to be an elbow at k=3.
Do not take this plot as the ultimate source of truth, but also look at your business needs - what if our business needs involve having a k between say 4 and 8?

There is also another metric that will help us decide.

#### Silhouette Score

In [ ]:
from sklearn.metrics import silhouette_score

K = range(2, 9)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_prep)
    silhouette.append(silhouette_score(X_prep, kmeans.predict(X_prep)))

print(list(K))
print(silhouette)

plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Score showing the optimal k')



You can see that the silhouette score also has an upward spike at 3! If our business requires us to have between 4 and 8 clusters, then according to this plot 5 would be the optimal number.